Installing libraries:

In [2]:
%load_ext autoreload
%autoreload 2

import os
import re
import sys
import numpy as np
import json
from pydub import AudioSegment, silence
import shutil
from IPython.display import Audio, display

rootdir_path = '/home/christosf/AttProtos'
sys.path.append(rootdir_path)
from apnet.datasets import MUSDB18

/home/christosf/anaconda3/envs/apnet2/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/christosf/anaconda3/envs/apnet2/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/christosf/anaconda3/envs/apnet2/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/christosf/anaconda3/envs/apnet2

In [3]:
# play audio file. If list, play all audio files
def play_audio(path_of_wav):
    if isinstance(path_of_wav, list):
        for item in path_of_wav:
            display(Audio(item))
    else:
        display(Audio(path_of_wav))

## MUSDB18 vocal tracks:


In [4]:
path = '/home/christosf/AttProtos/datasets/MUSDB18-w44100/'
os.chdir(path)

In [5]:
vocal_paths = []
for dirpath, dirnames, filenames in os.walk(path):
    for filename in filenames:
        if 'vocals.wav' == filename:
            print(dirpath + '/' + filename)
            vocal_paths.append(dirpath + '/' + filename)

/home/christosf/AttProtos/datasets/MUSDB18-w44100/original/test/Secretariat - Over The Top/vocals.wav
/home/christosf/AttProtos/datasets/MUSDB18-w44100/original/test/Lyndsey Ollard - Catching Up/vocals.wav
/home/christosf/AttProtos/datasets/MUSDB18-w44100/original/test/Moosmusic - Big Dummy Shake/vocals.wav
/home/christosf/AttProtos/datasets/MUSDB18-w44100/original/test/The Easton Ellises (Baumi) - SDRNR/vocals.wav
/home/christosf/AttProtos/datasets/MUSDB18-w44100/original/test/AM Contra - Heart Peripheral/vocals.wav
/home/christosf/AttProtos/datasets/MUSDB18-w44100/original/test/The Long Wait - Dark Horses/vocals.wav
/home/christosf/AttProtos/datasets/MUSDB18-w44100/original/test/Buitraker - Revo X/vocals.wav
/home/christosf/AttProtos/datasets/MUSDB18-w44100/original/test/The Easton Ellises - Falcon 69/vocals.wav
/home/christosf/AttProtos/datasets/MUSDB18-w44100/original/test/Sambasevam Shanmugam - Kaathaadi/vocals.wav
/home/christosf/AttProtos/datasets/MUSDB18-w44100/original/test/PR

In [8]:
vocal_paths[100:150]

['/home/christosf/AttProtos/datasets/MUSDB18-w44100/original/train/Jay Menon - Through My Eyes/vocals.wav',
 '/home/christosf/AttProtos/datasets/MUSDB18-w44100/original/train/Dreamers Of The Ghetto - Heavy Love/vocals.wav',
 '/home/christosf/AttProtos/datasets/MUSDB18-w44100/original/train/Lushlife - Toynbee Suite/vocals.wav',
 '/home/christosf/AttProtos/datasets/MUSDB18-w44100/original/train/Cnoc An Tursa - Bannockburn/vocals.wav',
 '/home/christosf/AttProtos/datasets/MUSDB18-w44100/original/train/Johnny Lokke - Whisper To A Scream/vocals.wav',
 '/home/christosf/AttProtos/datasets/MUSDB18-w44100/original/train/James May - If You Say/vocals.wav',
 '/home/christosf/AttProtos/datasets/MUSDB18-w44100/original/train/Music Delta - Beatles/vocals.wav',
 '/home/christosf/AttProtos/datasets/MUSDB18-w44100/original/train/BigTroubles - Phantom/vocals.wav',
 '/home/christosf/AttProtos/datasets/MUSDB18-w44100/original/train/Tim Taler - Stalker/vocals.wav',
 '/home/christosf/AttProtos/datasets/MUSD

In [6]:
vocals_isolated = []
for item in vocal_paths[100:150]:
    filename = item.split('/')[-2]
    vocals_isolated.append(filename)

In [25]:
vocals_in_mixtures = []
for item in vocal_paths[0:100]:
    filename = item.split('/')[-2]
    vocals_in_mixtures.append(filename)

In [38]:
path = '/home/christosf/AttProtos/datasets/MUSDB18Mixtures3/train/'
os.chdir(path)

In [63]:
to_keep = []
for dirpath, dirnames, filenames in os.walk(path):
    for filename in filenames:
        if '_vocals_' in filename:
            if filename.split('_')[0] in vocals_in_mixtures:
                to_keep.append(dirpath+filename)

In [65]:
len(to_keep)

670

In [1]:
# Choose the groundtruth vocals to replace the mixtures

In [14]:
path = '/home/christosf/AttProtos/datasets/MUSDB18Mixtures/vocaltracks/train/'
os.chdir(path)

In [15]:
to_keep_v = []
for dirpath, dirnames, filenames in os.walk(path):
    for filename in filenames:
        if '_vocaltrack_' in filename:
            if filename.split('_')[0] in vocals_isolated:
                to_keep_v.append(dirpath+filename)

In [16]:
len(to_keep_v)

643

In [95]:
path = '/home/christosf/AttProtos/datasets/MUSDB18Mixtures/train/'
os.chdir(path)

no_vocal_paths = []
for dirpath, dirnames, filenames in os.walk(path):
    for filename in filenames:
        if '_novocals_' in filename:
            no_vocal_paths.append(dirpath + '/' + filename)

In [96]:
len(no_vocal_paths)

1390

In [17]:
to_keep_v

['/home/christosf/AttProtos/datasets/MUSDB18Mixtures3/vocaltracks/train/Titanium - Haunted Age_vocaltrack_14.wav',
 '/home/christosf/AttProtos/datasets/MUSDB18Mixtures3/vocaltracks/train/Wall Of Death - Femme_vocaltrack_3.wav',
 '/home/christosf/AttProtos/datasets/MUSDB18Mixtures3/vocaltracks/train/Snowmine - Curfews_vocaltrack_8.wav',
 '/home/christosf/AttProtos/datasets/MUSDB18Mixtures3/vocaltracks/train/Music Delta - 80s Rock_vocaltrack_4.wav',
 '/home/christosf/AttProtos/datasets/MUSDB18Mixtures3/vocaltracks/train/AvaLuna - Waterduct_vocaltrack_26.wav',
 '/home/christosf/AttProtos/datasets/MUSDB18Mixtures3/vocaltracks/train/Clara Berry And Wooldog - Air Traffic_vocaltrack_4.wav',
 '/home/christosf/AttProtos/datasets/MUSDB18Mixtures3/vocaltracks/train/AvaLuna - Waterduct_vocaltrack_8.wav',
 '/home/christosf/AttProtos/datasets/MUSDB18Mixtures3/vocaltracks/train/Alexander Ross - Goodbye Bolero_vocaltrack_19.wav',
 "/home/christosf/AttProtos/datasets/MUSDB18Mixtures3/vocaltracks/train/

In [2]:
# Move and replace

In [18]:
for path in to_keep_v:
    src_path = path
    dest_path = path.replace('MUSDB18Mixtures/vocaltracks', 'MUSDB18Mixtures').replace('_vocaltrack_', '_vocals_')
    shutil.copy(src_path, dest_path)

### Create validation set if needed

In [29]:
import random

In [7]:
print(vocals_isolated)

['Jay Menon - Through My Eyes', 'Dreamers Of The Ghetto - Heavy Love', 'Lushlife - Toynbee Suite', 'Cnoc An Tursa - Bannockburn', 'Johnny Lokke - Whisper To A Scream', 'James May - If You Say', 'Music Delta - Beatles', 'BigTroubles - Phantom', 'Tim Taler - Stalker', 'Angela Thomas Wade - Milk Cow Blues', 'AvaLuna - Waterduct', 'The So So Glos - Emergency', 'The Districts - Vermont', 'Young Griffo - Facade', 'St Vitus - Word Gets Around', 'Voelund - Comfort Lives In Belief', 'Music Delta - Grunge', 'Auctioneer - Our Future Faces', 'Strand Of Oaks - Spacestation', 'Music Delta - Rock', 'Music Delta - Punk', 'Creepoid - OldTree', 'The Scarlet Brand - Les Fleurs Du Mal', 'Traffic Experiment - Once More (With Feeling)', 'Helado Negro - Mitad Del Mundo', 'Triviul - Dorothy', 'Leaf - Summerghost', 'Titanium - Haunted Age', 'Actions - One Minute Smile', "Actions - Devil's Words", 'Hollow Ground - Left Blind', 'Night Panther - Fire', 'Music Delta - Reggae', 'Skelpolu - Together Alone', 'Music D

In [30]:
#rc1 = random.choices(vocals_isolated, k=5)

In [5]:
# These

rc1 = ['Clara Berry And Wooldog - Waltz For My Victims',
 'Jay Menon - Through My Eyes',
 'Alexander Ross - Goodbye Bolero',
 'Traffic Experiment - Once More (With Feeling)',
 'Johnny Lokke - Whisper To A Scream']

"""
['Clara Berry And Wooldog - Waltz For My Victims',
 'Jay Menon - Through My Eyes',
 'Alexander Ross - Goodbye Bolero',
 'Traffic Experiment - Once More (With Feeling)',
 'Johnny Lokke - Whisper To A Scream']
 """

"\n['Clara Berry And Wooldog - Waltz For My Victims',\n 'Jay Menon - Through My Eyes',\n 'Alexander Ross - Goodbye Bolero',\n 'Traffic Experiment - Once More (With Feeling)',\n 'Johnny Lokke - Whisper To A Scream']\n "

In [4]:
rc1

['Clara Berry And Wooldog - Waltz For My Victims',
 'Jay Menon - Through My Eyes',
 'Alexander Ross - Goodbye Bolero',
 'Traffic Experiment - Once More (With Feeling)',
 'Johnny Lokke - Whisper To A Scream']

In [32]:
rc2 = random.choices(vocals_in_mixtures, k=10)

In [6]:
rc2 = ['We Fell From The Sky - Not You',
 'Actions - South Of The Water',
 'Girls Under Glass - We Feel Alright',
 'The Sunshine Garcia Band - For I Am The Moon',
 "Phre The Eon - Everybody's Falling Apart",
 'Meaxic - Take A Step',
 'Actions - South Of The Water',
 'Sambasevam Shanmugam - Kaathaadi',
 'The Long Wait - Back Home To Blue',
 'Motor Tapes - Shore']

In [6]:
rc2

['We Fell From The Sky - Not You',
 'Actions - South Of The Water',
 'Girls Under Glass - We Feel Alright',
 'The Sunshine Garcia Band - For I Am The Moon',
 "Phre The Eon - Everybody's Falling Apart",
 'Meaxic - Take A Step',
 'Actions - South Of The Water',
 'Sambasevam Shanmugam - Kaathaadi',
 'The Long Wait - Back Home To Blue',
 'Motor Tapes - Shore']

In [33]:
rc2

"""
['We Fell From The Sky - Not You',
 'Actions - South Of The Water',
 'Girls Under Glass - We Feel Alright',
 'The Sunshine Garcia Band - For I Am The Moon',
 "Phre The Eon - Everybody's Falling Apart",
 'Meaxic - Take A Step',
 'Actions - South Of The Water',
 'Sambasevam Shanmugam - Kaathaadi',
 'The Long Wait - Back Home To Blue',
 'Motor Tapes - Shore']
 """

['We Fell From The Sky - Not You',
 'Actions - South Of The Water',
 'Girls Under Glass - We Feel Alright',
 'The Sunshine Garcia Band - For I Am The Moon',
 "Phre The Eon - Everybody's Falling Apart",
 'Meaxic - Take A Step',
 'Actions - South Of The Water',
 'Sambasevam Shanmugam - Kaathaadi',
 'The Long Wait - Back Home To Blue',
 'Motor Tapes - Shore']

In [7]:
val_list = rc1 + rc2

In [8]:
val_list

['Clara Berry And Wooldog - Waltz For My Victims',
 'Jay Menon - Through My Eyes',
 'Alexander Ross - Goodbye Bolero',
 'Traffic Experiment - Once More (With Feeling)',
 'Johnny Lokke - Whisper To A Scream',
 'We Fell From The Sky - Not You',
 'Actions - South Of The Water',
 'Girls Under Glass - We Feel Alright',
 'The Sunshine Garcia Band - For I Am The Moon',
 "Phre The Eon - Everybody's Falling Apart",
 'Meaxic - Take A Step',
 'Actions - South Of The Water',
 'Sambasevam Shanmugam - Kaathaadi',
 'The Long Wait - Back Home To Blue',
 'Motor Tapes - Shore']

In [9]:
path = '/home/christosf/AttProtos/datasets/MUSDB18Mixtures3/train/'
os.chdir(path)

val_set = []
for dirpath, dirnames, filenames in os.walk(path):
    for filename in filenames:
        if filename.split('_')[0] in val_list:
            val_set.append(dirpath+filename)

In [12]:
val_set

['/home/christosf/AttProtos/datasets/MUSDB18Mixtures3/train/Jay Menon - Through My Eyes_novocals_2.wav',
 '/home/christosf/AttProtos/datasets/MUSDB18Mixtures3/train/Johnny Lokke - Whisper To A Scream_vocals_18.wav',
 '/home/christosf/AttProtos/datasets/MUSDB18Mixtures3/train/Jay Menon - Through My Eyes_novocals_8.wav',
 '/home/christosf/AttProtos/datasets/MUSDB18Mixtures3/train/Jay Menon - Through My Eyes_vocals_17.wav',
 '/home/christosf/AttProtos/datasets/MUSDB18Mixtures3/train/Traffic Experiment - Once More (With Feeling)_novocals_6.wav',
 '/home/christosf/AttProtos/datasets/MUSDB18Mixtures3/train/Jay Menon - Through My Eyes_novocals_5.wav',
 '/home/christosf/AttProtos/datasets/MUSDB18Mixtures3/train/Jay Menon - Through My Eyes_vocals_14.wav',
 '/home/christosf/AttProtos/datasets/MUSDB18Mixtures3/train/Jay Menon - Through My Eyes_vocals_29.wav',
 '/home/christosf/AttProtos/datasets/MUSDB18Mixtures3/train/Clara Berry And Wooldog - Waltz For My Victims_novocals_3.wav',
 '/home/christo

In [8]:
path = '/home/christosf/AttProtos/datasets/MUSDB18Mixtures3/validate/'
os.chdir(path)

val_set = []
for dirpath, dirnames, filenames in os.walk(path):
    for filename in filenames:
        if filename.split('_')[0] in val_list:
            val_set.append(dirpath+filename)

In [10]:
len(val_set)

280

In [11]:
import shutil

In [46]:
len(val_set)

280

In [12]:
for path in val_set:
    src_path = path
    dest_path = path.replace('validate', 'train')
    shutil.move(src_path, dest_path)

In [13]:
path = '/home/christosf/AttProtos/datasets/MUSDB18Mixtures3/features/MelSpectrogram/original/validate/'
os.chdir(path)

val_feat_set = []
for dirpath, dirnames, filenames in os.walk(path):
    for filename in filenames:
        if filename.split('_')[0] in val_list:
            val_feat_set.append(dirpath+filename)

In [14]:
for path in val_feat_set:
    src_path = path
    dest_path = path.replace('validate', 'train')
    shutil.move(src_path, dest_path)

### Detect non silent:

In [14]:
for path in vocal_paths:
    print(path) 
    nsil = []
    nsil_path = path.split('/vocals')[0] + "/non_silent.json"
    print(nsil_path)
    myaudio = AudioSegment.from_wav(path)
    dBFS=myaudio.dBFS
    nsil = silence.detect_nonsilent(myaudio, min_silence_len=1000, silence_thresh=dBFS-16)
    nsil = [((start),(stop)) for start,stop in nsil]
    print(nsil)
    with open(nsil_path, 'w') as f:
        json.dump(nsil, f)

/home/christosf/AttProtos/datasets/MUSDB18-w44100/original/test/Secretariat - Over The Top/vocals.wav
/home/christosf/AttProtos/datasets/MUSDB18-w44100/original/test/Secretariat - Over The Top/non_silent.json
[(28319, 33479), (35134, 39796), (41712, 47125), (48599, 53295), (55289, 56361), (58630, 67147), (84026, 89400), (90729, 95879), (97638, 103091), (104341, 109375), (111070, 112058), (114508, 122788), (130289, 134883), (137112, 140217), (143723, 147954), (150080, 157237), (187118, 188259), (190519, 198928), (202334, 204060), (205803, 207898), (209020, 214096)]
/home/christosf/AttProtos/datasets/MUSDB18-w44100/original/test/Lyndsey Ollard - Catching Up/vocals.wav
/home/christosf/AttProtos/datasets/MUSDB18-w44100/original/test/Lyndsey Ollard - Catching Up/non_silent.json
[(14371, 18183), (20647, 23416), (27020, 30649), (33270, 36082), (39677, 42697), (45773, 49990), (52295, 55707), (58407, 61603), (64887, 68651), (71230, 74170), (77158, 81917), (83707, 86910), (90207, 93305), (96460,